<a href="https://colab.research.google.com/github/thamizhannal/eva5/blob/master/EVA5_Session_6_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Target:
Reach 99.4% accuracy consistently in last 3-4 epochs by a applying more regularization in the form of dropout and increase LR rate to understand if convergence occurs faster and use LR schedular to change LR wherever Loss gets to plateau

###Results:
1. Parameters: 7,917
2. Best Train Accuracy: 99.08%
3. Best Test Accuracy: 99.43% 
4. greater than 99.4% in last 4 epochs(12-15)

###Analysis:
1. Dropout added the necessary regularization required to make model robust
2. Though increased LR was helping in fast convergence but after 5-6 epochs it would tapper off implying a need for smaller LR so used a step LR schedule at after 4 epochs and decreased it to 0.8 times the previous LR

###All the changes added to the base CNN architechture to acheive the target accuracy 
1. Input image normalization
2. Image augmentation - rotation for Train data 
3. Batch Normalization
4. Dropouts 
5. LR scheduler 
6. GAP

 

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class GhostBatchNorm(BatchNorm):
    def __init__(self, num_features, num_splits, **kw):
        super().__init__(num_features, **kw)
        self.num_splits = num_splits
        self.register_buffer('running_mean', torch.zeros(num_features * self.num_splits))
        self.register_buffer('running_var', torch.ones(num_features * self.num_splits))

    def train(self, mode=True):
        if (self.training is True) and (mode is False):  # lazily collate stats when we are going to use them
            self.running_mean = torch.mean(self.running_mean.view(self.num_splits, self.num_features), dim=0).repeat(
                self.num_splits)
            self.running_var = torch.mean(self.running_var.view(self.num_splits, self.num_features), dim=0).repeat(
                self.num_splits)
        return super().train(mode)

    def forward(self, input):
        N, C, H, W = input.shape
        if self.training or not self.track_running_stats:
            return F.batch_norm(
                input.view(-1, C * self.num_splits, H, W), self.running_mean, self.running_var,
                self.weight.repeat(self.num_splits), self.bias.repeat(self.num_splits),
                True, self.momentum, self.eps).view(N, C, H, W)
        else:
            return F.batch_norm(
                input, self.running_mean[:self.num_features], self.running_var[:self.num_features],
                self.weight, self.bias, False, self.momentum, self.eps)

In [ ]:
#Receptive field is calcualted based on actual formula
class Net(nn.Module):
    def __init__(self, enable_bn=True ):
        super(Net, self).__init__()
        self.enable_bn = enable_bn

        #block1
        self.conv1 = nn.Conv2d(1, 8, 3,padding = True) #o/p size:28; rf: 3
        self.Batchnorm1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 15, 3) #o/p size: 26; rf: 5
        self.Batchnorm2 = nn.BatchNorm2d(15)


        #transition block
        self.pool1 = nn.MaxPool2d(2, 2) #o/p size: 13; rf: 6
        self.pool1trns = nn.Conv2d(15, 10, 1)#o/p size: 13; rf: 6
        self.Batchnormtrns1 = nn.BatchNorm2d(10)
        
        #block2
        self.conv3 = nn.Conv2d(10,14,3) #o/p size: 11; rf: 10
        self.Batchnorm3 = nn.BatchNorm2d(14)
        self.dp3 = nn.Dropout(p = 0.10) 
        self.conv4 = nn.Conv2d(14, 16, 3) #o/p size: 9; rf: 14
        self.Batchnorm4 = nn.BatchNorm2d(16)
        self.dp4 = nn.Dropout(p = 0.10)
        self.conv5 = nn.Conv2d(16, 20, 3) #o/p size: 7; rf: 18
        self.Batchnorm5 = nn.BatchNorm2d(20)
        self.dp5 = nn.Dropout(p = 0.10)

        #gap and 1X1 
        self.conv6_avgp = nn.AvgPool2d(kernel_size=7) #o/p size: 1; rf: 30
        self.pool2trns = nn.Conv2d(20,10,1) #6


    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        if self.enable_bn:
          x = self.Batchnorm1(x)

        x = self.conv2(x)
        x = F.relu(x)
        if self.enable_bn:
          x = self.Batchnorm2(x)


        x = self.pool1(x)
        x = self.pool1trns(x)
        x = F.relu(x)
        if self.enable_bn:
          x = self.Batchnormtrns1(x)

        x = self.conv3(x)
        x = F.relu(x)
        if self.enable_bn:
          x = self.Batchnorm3(x)
        x = self.dp3(x)
 
        x = self.conv4(x)
        x = F.relu(x)
        if self.enable_bn:
          x = self.Batchnorm4(x)
        x = self.dp4(x)

        x = self.conv5(x)
        x = F.relu(x)
        if self.enable_bn:
          x = self.Batchnorm5(x)
        x = self.dp5(x)

        x = self.conv6_avgp(x)

        x = self.pool2trns(x)


        x = x.view(-1, 10)
        return F.log_softmax(x,dim=-1)

In [ ]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net(True).to(device)

summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3           [-1, 15, 26, 26]           1,095
       BatchNorm2d-4           [-1, 15, 26, 26]              30
         MaxPool2d-5           [-1, 15, 13, 13]               0
            Conv2d-6           [-1, 10, 13, 13]             160
       BatchNorm2d-7           [-1, 10, 13, 13]              20
            Conv2d-8           [-1, 14, 11, 11]           1,274
       BatchNorm2d-9           [-1, 14, 11, 11]              28
          Dropout-10           [-1, 14, 11, 11]               0
           Conv2d-11             [-1, 16, 9, 9]           2,032
      BatchNorm2d-12             [-1, 16, 9, 9]              32
          Dropout-13             [-1, 16, 9, 9]               0
           Conv2d-14             [-1, 2

In [ ]:
torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation((-8.0, 8.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch, 
          enable_l1=False, enable_l2=False, enable_gbn=False):
    model.train()
    #pbar = tqdm(train_loader)
    train_loss = 0 
    train_correct = 0
    # For best lambda value, apply grid search
    lambda_l1 = 0.01
    #scheduler.step()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)

        # loss function
        # L1 loss
        if (enable_l1 and not (enable_l2 and enable_gbn)):
            #print ("LOSS: l1")
            l1_loss = 0
            for param in model.parameters():
                l1_loss = l1_loss + param.abs().sum()
            loss = loss + lambda_l1 * l1_loss
        # L2 loss    
        #elif enable_l2 and not (enable_l1 and enable_gbn):
            #print ("LOSS:l2")
        # L1L2 loss
        elif enable_l1 and enable_l2 and not enable_gbn:
            print("l1l2")
        # GBN only
        elif enable_gbn and not (enable_l1 and enable_l2) :  
            print("gbn only")
        # L1L2&GBN loss
        elif enable_gbn and enable_l1 and enable_l2 :  
            print("l1l2gbn")
        #

        loss.backward()
        optimizer.step()
        #pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        train_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        train_correct += pred.eq(target.view_as(pred)).sum().item()

    train_loss /= len(test_loader.dataset)
    print('Epoch: {:.0f},LR: {}.\nTrain set: train Average loss: {:.4f}, train_Accuracy: {}/{} ({:.4f}%)\n'.format(
        epoch,optimizer.param_groups[0]['lr'],train_loss, train_correct, len(train_loader.dataset),
        100. * train_correct / len(train_loader.dataset)))
        

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss

            pred_test = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred_test.eq(target.view_as(pred_test)).sum().item()

    test_loss /= len(test_loader.dataset)

    
    print('Test set: test Average loss: {:.4f}, test Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    


In [ ]:
from torch.optim.lr_scheduler import StepLR

# Loop To Run Network 
# 1. with L1 + BN
# 2. with L2 + BN
# 3. with L1 and L2 with BN
# 4. with GBN
# 5. with L1 and L2 with GBN

losses = ['l1', 'l2', 'l1&l2', 'gbn', 'l1l2gbn']

for idx in range(2): # len(losses)
    i = idx+1
    enable_l1 = False
    enable_l2 = False
    enable_gbn = False

    print("{} {} loss".format((i),losses[i]))
    if i == 1 or i==2 or i==3:
        model = Net(enable_bn=True).to(device)
        print("BN enabled for option: {}".format(i))
    # DisableBN for GBN & L1&L2 with GBN
    elif i==4 or i==5:
        model = Net(enable_bn=False).to(device)
        print("BN disabled for option: {}".format(i))
    
    optimizer = optim.SGD(model.parameters(), lr=0.03, momentum=0.9)
    if i==2 or i==3 or i==5:
        decay = 0.001
        optimizer = optim.SGD(model.parameters(), lr=0.03, momentum=0.9, weight_decay=decay)  
        print("Weight decay {} set for option: {}".format(decay, i))
    
    scheduler = StepLR(optimizer, step_size=4, gamma=0.8)
    
    if i==1:
         enable_l1 = True
         print("LOSS: l1 loss enabled")
    elif i==2:
         enable_l2 = True
         print("LOSS: l2 loss enabled")
    elif i==3:
         enable_l1 = True
         enable_l2=True
         print("LOSS: l1l2 loss enabled")
    elif i==4:         
         enable_gbn = True
         print("LOSS: gbn loss enabled")
    elif i==5:
         enable_l1 = True
         enable_l2=True
         enable_gbn = True
         print("LOSS: l1l2gbn loss enabled")


    for epoch in range(1, 16):
        train(model, device, train_loader, optimizer, epoch, 
              enable_l1=enable_l1, enable_l2=enable_l2, enable_gbn=enable_gbn)
        scheduler.step()
        test(model, device, test_loader)





2 l2 loss
BN enabled for option: 1
LOSS: l1 loss enabled
Epoch: 1,LR: 0.03.
Train set: train Average loss: 3.8450, train_Accuracy: 49099/60000 (81.8317%)

Test set: test Average loss: 1.2385, test Accuracy: 5948/10000 (59.4800%)

Epoch: 2,LR: 0.03.
Train set: train Average loss: 3.0888, train_Accuracy: 50931/60000 (84.8850%)

Test set: test Average loss: 1.7729, test Accuracy: 5015/10000 (50.1500%)

Epoch: 3,LR: 0.03.
Train set: train Average loss: 3.1230, train_Accuracy: 50845/60000 (84.7417%)

Test set: test Average loss: 2.5070, test Accuracy: 3967/10000 (39.6700%)

Epoch: 4,LR: 0.03.
Train set: train Average loss: 3.0975, train_Accuracy: 50961/60000 (84.9350%)

Test set: test Average loss: 1.0903, test Accuracy: 6511/10000 (65.1100%)

Epoch: 5,LR: 0.024.
Train set: train Average loss: 3.0534, train_Accuracy: 51007/60000 (85.0117%)

Test set: test Average loss: 2.4221, test Accuracy: 4444/10000 (44.4400%)

Epoch: 6,LR: 0.024.
Train set: train Average loss: 3.0338, train_Accuracy: 51

In [ ]:
losses = ['l1', 'l2', 'l1&l2', 'gbn', 'l1l2gbn']

for i in range(len(losses)):
    print("{} {} loss".format((i+1),losses[i]))



In [ ]:
enable_l1=True
enable_l2=True
enable_gbn=False
if enable_l1 and not enable_l2 and not enable_gbn:
  print ("l1")
elif not enable_l1 and enable_l2 and not enable_gbn:
  print ("l2")
elif enable_l1 and enable_l2 and not enable_gbn:
  print("l1l2")
elif enable_gbn and not enable_l1 and not enable_l2 :  
  print("gbn only")
elif enable_gbn and enable_l1 and enable_l2 :  
  print("l1l2gbn")

In [ ]:
    i=5

    if i==1:
         enable_l1 = True
         print("LOSS: l1 loss enabled")
    elif i==2:
         enable_l2 = True
         print("LOSS: l2 loss enabled")
    elif i==3:
         enable_l1 = True
         enable_l2=True
         print("LOSS: l1l2 loss enabled")
    elif i==4:         
         enable_gbn = True
         print("LOSS: gbn loss enabled")
    elif i==5:
         enable_l1 = True
         enable_l2=True
         enable_gbn = True
         print("LOSS: l1l2gbn loss enabled")
